### 大規模言語モデルの基礎

#### GPTをTransformersで使う

In [1]:
import os
os.environ['HF_HOME'] = "/workspaces/LLM/huggingface_cache"
cache_dir = "/workspaces/LLM/huggingface_cache"

In [2]:
from transformers import pipeline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import file_utils

# 現在のキャッシュディレクトリを取得
cache_dir = file_utils.default_cache_path
print(f"Current HuggingFace cache directory: {cache_dir}")

Current HuggingFace cache directory: /workspaces/LLM/huggingface_cache/transformers


In [8]:
# 後続するテキストを予測するpipelineを作成
generator = pipeline(
    "text-generation", model="abeja/gpt2-large-japanese",
    cache_dir=cache_dir
)
# "日本で一番高い山は"に続くテキストを生成
outputs = generator("日本の総理大臣の名前は")
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


日本の総理大臣の名前はすべてカタカナになっている。 カタカナをローマ字に直して読み替えれば「ザ」の発音に変わる。 英語でも同じ。 本来、漢字表記に「ザ」の字が使われているため、この名に統一されている。 


In [5]:
### ファインチューニング
### BERTモデル（エンコーダーモデル）
# [MASK]部分の予測
import pandas as pd

# マスクされたトークンを予測するパイプライン
fill_mask = pipeline(
    "fill-mask", model="tohoku-nlp/bert-base-japanese-v3"
)
masked_text = "日本の首都は[MASK]である"
outputs = fill_mask(masked_text)
# 上位3件をテーブル表示
display(pd.DataFrame(outputs[:3]))

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-v3 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,score,token,token_str,sequence
0,0.884170,12569,東 京,日本 の 首都 は 東京 で ある
1,0.024820,12759,大 阪,日本 の 首都 は 大阪 で ある
2,0.020864,13017,京 都,日本 の 首都 は 京都 で ある


In [4]:
masked_text = "今日の映画は刺激的で面白かった。この映画は[MASK]。"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:3]))

,score,token,token_str,sequence
0,0.683933,23845,素 晴 ら し い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 素晴らしい 。
1,0.101235,24683,面 白 い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 面白い 。
2,0.048003,26840,楽 し い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 楽しい 。


In [4]:
### T5モデル（エンコーダー・デコーダーモデル）
# text-to-textで生成するpipelineを作成
t2t_generator = pipeline(
    "text2text-generation", model="retrieva-jp/t5-large-long",
)
# マスクされたスパンを予測
masked_text = "江戸時代を開いたのは、<extra_id_0>である。"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

徳川家康


In [3]:
# トークンIDを表示
t2t_generator.tokenizer.convert_tokens_to_ids("<extra_id_1>")

32098

In [4]:
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

日本銀行


In [5]:
# "日本銀行"はモデルの語彙に存在しない
"日本銀行" in t2t_generator.tokenizer.vocab

False

### Tokenizer

In [6]:
### バイト対符号化
# 単語とその頻度
word_freqs = {
    "たのしい": 6,
    "たのしさ": 2,
    "うつくしい": 4,
    "うつくしさ": 1,
}
# 語彙を文章で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))
# 単語とその分割状態
splits = {word: [char for char in word] for word in word_freqs}

In [10]:
# サブワードの組を計算する関数
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
) -> tuple[str, str]:
    """
    最も頻度の高い隣接するサブワードの組を計算する
    """
    pair_freqs = Counter() # サブワードの組のカウンター
    for word, freq in word_freqs.items(): # すべての単語を処理
        split = splits[word] # 現在の単語の分割状況を取得
        # すべての隣接するサブワードの組を処理
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            # サブワードの組の頻度に単語の頻度を加算
            pair_freqs[pair] += freq
    # カウンタから最も頻度の高いサブワードの組を取得
    pair, _ = pair_freqs.most_common(1)[0]
    return pair

def merge_pair(
    target_pair: tuple[str, str], splits: dict[str, list[str]]
) -> dict[str, list[str]]:
    """
    サブワードの組を結合する
    """
    l_str, r_str = target_pair
    for word in word_freqs: # すべての単語を処理
        split = splits[word] # 現在の単語の分割状況を取得
        i = 0
        while i < len(split) - 1:
            # サブワードの組が結合対象と一致したら結合
            if split[i] == l_str and split[i + 1] == r_str:
                split = split[:i] + [l_str + r_str] + split[i + 2 :]
            i += 1
        splits[word] = split # 現在の結合状態を更新
    return splits

In [12]:
for step in range(9):
    # 最も頻度の高い隣接するサブワードの組を計算
    target_pair = compute_most_frequent_pair(splits)
    # サブワードの組を結合
    splits = merge_pair(target_pair, splits)
    # 語彙にサブワードの組を追加
    vocab.append(target_pair)

In [13]:
print(vocab)

['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]
